# This is a Notebook to run RNAseq analysis server
# For FFPE or frozen samples 


This is a warp up of the few steps for the pre-procesing in the RNAseq analaysis, the notebook include:
- fastqc
- trimming, trim-galore
- Alignment, STAR
- Annotation counts, featureCounts
- multiqc, QC of the process



In [2]:
#this is in case you forget where you are and what is in it
!pwd
#!ls

/DATA/home/l.gonzalez/Notebooks


In [1]:
import os, sys
import glob
import logging

### Before starting the analysis we will check the QC of the fastq files usign fastqc
To do that, please go to the cell below and change the path to the folder where the fastq files are, then run it

In [ ]:
!fastqc /DATA/share/pipelines/lgleon/test/testCode_steps_Nousefuldata/testdata/fastq/*fastq.gz

## Run trim_galore for the trimming part paired-end
Then remove the adaptors using trim galore (conda)
The CLI can be run directly 

In [1]:
#!conda init bash

In [ ]:
#Trimming step go where the fastq raw files are

#Binary = "/path/TrimGalore-0.6.5/trim_galore"
#Be aware that the path to the raw files (prj) is hard coded
# Change it  !!!!!!!!!!!!
#prj is hardcode, need to be change with any new analysis

#Script, DO NOT RUN !!!!!!!!!!!!!!!!!
#!bin/bash

#############################################################################
##############################################################################
# SINGLE-END part
##############################################################################
##############################################################################

##careful with fastq.gz OR fq.gz   can give problems!

##Hardcode fastq folder change when need it
#    prj=/path/C5_RNAseq_FFPE/fastq_files/

#for i in ${prj}/*R1*.f*q.gz 

#do 
#    name=$(basename ${i} _R1_001.fastq.gz)

#	echo ${name}
#	echo ${name}_R1_001.fastq.gz
#	echo ${name}_R2_001.fastq.gz
#	echo ${prj}trimmed/


#    /path/programs/TrimGalore-0.6.5/trim_galore -q 35 -o ${prj}trimmed/ ${name}_R1_001.fastq.gz
    

#done

#############################################################################
##############################################################################
# PAIRED-END part
##############################################################################
##############################################################################
#to work with this part remove the # in the single ones, doubles are explanations

##careful with fastq.gz OR fq.gz   can give problems!

##Hardcode fastq folder change when need it
        prj=/path/organoids_rawdata/RNAseq/fastq_normals

for i in ${prj}/*R1*.f*q 

do 
    #name=$(basename ${i} _R1_001.fastq.gz)
    name=$(basename ${i/%_R1_001.f*q})

	echo ${name}
    #echo ${name/%_R1_001.f*q}       #filenamestem=${file/%.f*q.gz/}
	echo ${name}_R1_001.fastq  #option namefq1=${name}_R1_001.fastq.gz, then in the cli use ${namefq1}
	echo ${name}_R2_001.fastq  #option namefq2=${name}_R2_001.fastq.gz
    echo ${prj}/${name}_R1_001.fastq
    echo ${prj}/trimmNormals/          #option pathoutput=${prj}/trimmed/



    /path/programs/TrimGalore-0.6.5/trim_galore -q 35 --paired -o ${prj}/trimmNormals/ ${prj}/${name}_R1_001.fastq ${prj}/${name}_R2_001.fastq #> logtrim.txt &
    
    
    #### If you run this from Notebook, to the other folder then ${name} is not enough, that is only the name os the files
    ####you need to put ${prj}/${name}_R1_001.fastq.gz to work ??
    


done


In [2]:
#once you change the prj folder, run this cell
#!bash /<path>/testrun_trimgalorePaired_notebook.sh

!bash /path/Notebooks/testrun_trimgalorePaired_notebook.sh


### Run the aligner (STAR)
In this case is a small script to run STAR with paired-end files
Be aware of the path to the directory to change for the new one

In [25]:
# Variables to know where are the binary, references DO NOT CHANGE them
STAR_BINARY="/path/programs/STAR-2.7.5c/bin/Linux_x86_64_static/STAR"
GENE_REFERENCE="/path/References/Ensembl/STAR_Ref/Homo_sapiens.GRCh38.99.gtf"
STAR_INDEX="/path/References/Ensembl/newSTAR_idx"
USE_THREADS=16

#those ones you need to change with each analysis (run), here and in the script before run it
WORKING_DIR = "/path/C5_RNAseq_FFPE/"
DIR_INPUT_SAMPLES = "/path/C5_RNAseq_FFPE/fastq_files"
DIR_BAM_OUTPUT = "/path/C5_RNAseq_FFPE/bam_test_notrimm/"


In [ ]:
# The idea is run directly the small bash script /path/Notebooks/run_newpairedSTAR.sh
# OR  /home/l.gonzalez/Notebooks/run_STAR_singleORpaired.sh
# what is in this scrript: DO NOT RUN IT, IT WILL NOT WORK LIKE THAT

#script:

#!bin/bash

#############################################################################
##############################################################################
# SINGLE-END part
##############################################################################
##############################################################################


#where are the data that we are analyzing, this part is hard code need to be change each time
prj=/path/organoids_rawdata/RNAseq/test-code

#loop to interact over all the fastq files in the folder, take one file
#check the nale-tail of the files and adapt if need it
for i in ${prj}/*R1*.f*q.gz 

do 
    name=$(basename ${i} _R1_001.fastq.gz) 

	echo ${name}
	echo ${name}_R1_001.fastq.gz
#	echo ${name}_R2_001.fastq.gz
	echo ${prj}/output/


    /home/l.gonzalez/programs/STAR-2.7.5c/bin/Linux_x86_64_static/STAR \
    --runMode alignReads \
    --genomeDir /path/lgleon/References/Ensembl/newSTAR_idx/ \
    --runThreadN 16 \
    --sjdbGTFfile /path/References/Ensembl/STAR_Ref/Homo_sapiens.GRCh38.99.gtf \
    --readFilesIn  ${name}_R1_001.fastq.gz \
    --outSAMtype BAM SortedByCoordinate \
    --outFileNamePrefix ${prj}/output/ \
    --outFilterMismatchNmax 10 \
    --outFilterIntronMotifs None \
    --chimSegmentMin 1 \
    --sjdbScore 2 \
    --quantMode GeneCounts \
    --readFilesCommand zcat


done

# you can convert this one in parired end, 
# simple removing the # from the R2 and adding the second fastq in the input

### To modified the file here, just open the sh file and do it, easy peasy 
We are in /home/l.gonzalez/Notebooks

In [3]:
## Alignment with STAR
## Go to the folder where you wanto to save the results
## Once you changed the proj folder, run this cell

#!bash /home/l.gonzalez/Notebooks/run_newpairedSTAR.sh 
#!bash /home/l.gonzalez/Notebooks/run_STAR_singleORpaired.sh

#testing code 
#!bash /DATA/share/pipelines/lgleon/organoids_rawdata/RNAseq/test-code/test_newpairedSTAR.sh
!bash /path/Notebooks/run_newpairedSTAR.sh


### Run featureCounts
Again it is just a line in CLI, so go where you have the bam files and run it from there

In [4]:
## To run featureCounts is also a line that we can call frem here
## go to the folder where all the bam files are store after algnment 
## Be aware:
Binary = "/path/programs/subread-2.0.1-source/bin/featureCounts"
GFT_Ref = "/path/References/gencoderef/hg19/gencode.v19.annotation.gtf"

## run
!/path/programs/subread-2.0.1-source/bin/featureCounts \
-T 16 -t gene -g gene_name -O -s 0 -M -Q 10 -a \
/path/References/Ensembl/STAR_Ref/Homo_sapiens.GRCh38.99.gtf -F GTF/GFF \
-o /path/organoids_rawdata/RNAseq/counts/fcounts_hg38_genename.counts \
/path/organoids_rawdata/RNAseq/bam_star/*bam

#!/path/programs/subread-2.0.1-source/bin/featureCounts \
#-T 16 -t exon -g exon_id -O -s 0 -M -Q 10 -a \
#/path/References/Ensembl/STAR_Ref/Homo_sapiens.GRCh38.99.gtf -F GTF/GFF \
#-o /path/NGS_protocol/RNA_Seq/Processed_Data/counts/NGS_normal_exon.counts \
#/path/NGS_protocol/RNA_Seq/Processed_Data/complete/Bam_normal_adjcentColon/*bam

#Feature type(-t), 3rd column GTF file
#CDS
#Selenocysteine
#exon
#five_prime_utr
#gene
#start_codon
#stop_codon
#three_prime_utr
#transcript

#Atrivute that can be use (-g):
#An example of attributes included in your GTF annotation is:
#'gene_id "ENSG00000223972"; 
#gene_version "5"; 
#transcript_id "ENST00000456328"; 
#transcript_version "2"; 
#exon_number "1"; 
#gene_name "DDX11L1"; 
#gene_source "havana"; 
#gene_biotype "transcribed_unprocessed_pseudogene"; 
#transcript_name "DDX11L1-202"; 
#transcript_source "havana"; 
#transcript_biotype "processed_transcript"; 
#exon_id "ENSE00002234944"; 
#exon_version "1"; 
#tag "basic"; 
#transcript_support_level "1";' 


### After the analysis we will check the QC of the steps that we ran
To do that, please go to the cell below and change the path to the folders where the fastqc files are, also the bam/bai files and multiqc will create a html with all the files

In [ ]:
!multiqc /path/test/testCode_steps_Nousefuldata/testdata/fastq/

In [ ]:
#for fname in glob.glob(DIR_INPUT_SAMPLES + "/*.f*q.gz"):
#    print(fname)